In [ ]:
!pip install pyarrow
!pip install BackTranslation

In [2]:
import re
import pandas as pd
from BackTranslation import BackTranslation

In [32]:
#source: https://ansm.sante.fr/documents/reference/repertoire-des-medicaments-generiques
#create drugs.txt

res = []
with open("data/liste_medicaments.txt", encoding = 'utf-8') as f:
    for line in f:
        if len(line) >= 22 and line[:22] == 'Dénomination commune :':
            line = line[23:]
            n_medicament = line.split(' Voie')[0]
            n_medicament = re.sub('\(.*?\)', '', n_medicament)
            n_medicament = n_medicament.split('+')
            for medicament in n_medicament:
                medicament = "".join([x if (x.isalpha() or x.isspace()) else '' for x in medicament]).strip().lower()
                medicament = " ".join(list(dict.fromkeys(medicament.split(" "))))
                res.append(medicament)
medic_list_propre = pd.DataFrame(set(res), columns = ["medicament"])
medic_list_propre.to_csv("data/drugs.txt", index=False)

In [6]:
#create back_translation dictionnaries

import time
from collections import defaultdict
df = pd.read_parquet("data/tokens_sans_bert.parquet")
trans = BackTranslation(url=[
      'translate.google.com',
      'translate.google.co.kr',
    ], proxies={'http': '127.0.0.1:1234', 'http://host.name': '127.0.0.1:4012'})
id_add = 101
df_base_sentence = df[df["id_phrase"] % 1000 == 0]
dict_sentence = defaultdict(lambda : "")
dict_labels = defaultdict(lambda : [])
cur_phrase_part = ""
cur_phrase_translated = ""
cur_id_phrase = 1000
for index, row in df_base_sentence.iterrows():
    if row["label"] == 'O' and row["id_phrase"] == cur_id_phrase:
        cur_phrase_part += row["token"] + " "
    else:
        if cur_phrase_part != "":
            try:
                time.sleep(2)
                cur_phrase_translated += trans.translate(cur_phrase_part, src='fr', tmp='en').result_text
                cur_phrase_part = ""
            except:
                print("error with: " + cur_phrase_part)
        if row["id_phrase"] != cur_id_phrase:
            dict_sentence[cur_id_phrase + id_add] = cur_phrase_translated + " "
            cur_id_phrase = row["id_phrase"]
            cur_phrase_translated = ""
        if row["label"] != 'O':
            dict_labels[cur_id_phrase + id_add].append([row['label'].split('-')[1], len(cur_phrase_translated), len(cur_phrase_translated) + len(row["token"])])
            cur_phrase_translated += row["token"] + " "
        else:
            cur_phrase_part += row["token"]
            
    

error with: une 
error with: une 
error with: une 
error with: une 
error with: une 
error with: une 
error with: manquants. 
error with: une 
error with: une 
error with: irrégulières 


In [ ]:
#create back_translation dataset

import json

with open("data/back_translated.jsonl", 'w', encoding='utf8') as f:
    for id_phrase in dict_sentence:
        f.write(json.dumps({"id": id_phrase, "text": dict_sentence[id_phrase], "labels": dict_labels[id_phrase], "Comments": []}, ensure_ascii=False) + "\n")